## Problem
- How to inspect the parameters and return types of a function before making the actual call?

In [1]:
def calculator(a, b, operator='+'):
    operations = {
        '+': lambda a, b: a+b,
        '-': lambda a, b: a-b,
        '*': lambda a, b: a*b,
        '/': lambda a, b: a/b,
    }
    
    operation = operations[operator]
    result = operation(a, b)
    return result


def calculator_annot(a:float, b:float, operator:"str in ('+', '-', '*', '/')"='+') -> float: #<5>
    operations = {
        '+': lambda a, b: a+b,
        '-': lambda a, b: a-b,
        '*': lambda a, b: a*b,
        '/': lambda a, b: a/b,
    }
    
    operation = operations[operator]
    result = operation(a, b)
    return result

## Answer
- Use the inspect module

In [2]:
# before looking at inspect module let's see something cool
print(dir(calculator))
print()
print(dir(calculator_annot))

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [5]:
print(calculator.__defaults__) #<0>
print(calculator_annot.__defaults__)

('+',)
('+',)


In [11]:
print(calculator.__code__.__dir__())

['__repr__', '__hash__', '__getattribute__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__new__', '__sizeof__', 'co_argcount', 'co_kwonlyargcount', 'co_nlocals', 'co_stacksize', 'co_flags', 'co_code', 'co_consts', 'co_names', 'co_varnames', 'co_freevars', 'co_cellvars', 'co_filename', 'co_name', 'co_firstlineno', 'co_lnotab', '__doc__', '__str__', '__setattr__', '__delattr__', '__init__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__dir__', '__class__']


In [21]:
print(calculator.__code__.co_argcount) #<1>
print(calculator.__code__.co_kwonlyargcount) #<2>
print(calculator.__code__.co_varnames) #<3>

3
0
('a', 'b', 'operator', 'operations', 'operation', 'result')


In [22]:
# getting a function signature using the inspect module

from inspect import signature

sign = signature(calculator) #<4>
sign_annot = signature(calculator_annot) #<4>

In [38]:
for name, param in sign.parameters.items():
    print(param.kind, ':', name, '=', param.default)
print(sign.return_annotation)

POSITIONAL_OR_KEYWORD : a = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : b = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : operator = +
<class 'inspect._empty'>


In [39]:
for name, param in sign_annot.parameters.items():
    annotation = repr(param.annotation).ljust(30)
    print(param.kind, ': ',  annotation, ':', name, '=', param.default)
print(sign_annot.return_annotation)

POSITIONAL_OR_KEYWORD :  <class 'float'>                : a = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD :  <class 'float'>                : b = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD :  "str in ('+', '-', '*', '/')"  : operator = +
<class 'float'>


In [32]:
# checking if a function call is possible before making the actual call

args_call1 = (1, 2, '+') # call with params as positional args
bound1 = sign.bind(*args_call1) #<5>
for name, value in bound1.arguments.items():
    print(name, '=', value)

a = 1
b = 2
operator = +


In [36]:
args_call2 = {'a': 1, 'b': 2, 'operator': '+'} # call with params as keyword args
bound2 = sign.bind(**args_call2) 
for name, value in bound2.arguments.items():
    print(name, '=', value)
    
print('---')

del args_call2['operator'] # deleting operator should be fine since there is a default parameter
bound2 = sign.bind(**args_call2)
for name, value in bound2.arguments.items():
    print(name, '=', value)

a = 1
b = 2
operator = +
---
a = 1
b = 2


In [40]:
args_call3 = {'b': 2, 'operator': '+'} # removing a from the arguments would make the call impossible as a is required.
bound3 = sign.bind(**args_call3) #<6> # as expected this raises an exception

TypeError: missing a required argument: 'a'

In [42]:
calculator(**args_call3) # calling the function for real with the same set of arguments raises the same exception

TypeError: calculator() missing 1 required positional argument: 'a'

## Discussion
- <0> returns the values of all the parameters with default values
- <1> returns the number of position and keyword arguments of the function
- <2> returns only the number of keyword arguments
- <3> returns all the parameters and variables defined within the function
- <4> constructs a signature object from a function
- <5> binds the arguments provided with and check if they are valid arguments, if a call to the function could be made with those arguments returns a bound object.
- <6> when the call the function can't be made with the provided arguments, bind throws the same exception calling the actual function with the same arguments would have thrown.